<a href="https://colab.research.google.com/github/Pushpit07/Tech-Rush_AI-Ranking/blob/main/Tech_Rush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement 2

Sears is a pioneer in the retail industry and operates in a multitude of domains, selling millions of products on a daily basis and continuously adding thousands of new products to its product line.
In order to effectively evaluate the performance of retail stores, it is essential to have a consistent analysis system in place factoring the local market and competition.

Given a list of store location data, build a system that will generate a ranking of the locations based on the following -

   -- Distance from nearest Walmart, Costco, Target, and Home Depot stores. Each of these distances will have its own weightage that has to be accounted for in the ranking algorithm.
   
   -- Population density in and around the location.
   
   -- Economy around the location

The weightage of these parameters needs to be configurable in the system. 

## Installation & Imports

In [112]:
%%capture
!pip install uszipcode

import pandas as pd
import numpy as np
import requests
import warnings
warnings.filterwarnings("ignore")

try:
  from uszipcode.search import SearchEngine
  search = SearchEngine()
except:
  pass

## Loading the dataset

In [113]:
# Load store location data
data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/locn_data.csv")

df = data[["locn_nbr", "locn_cty", "locn_st_cd", "zip_cd", "zip_pls_4", "locn_addr"]]
df.head()

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST


In [114]:
df.count()

locn_nbr      99
locn_cty      99
locn_st_cd    99
zip_cd        99
zip_pls_4     99
locn_addr     99
dtype: int64

## Adding coordinates, population density, and economy data for locations given in the dataset

In [115]:
START_INDEX = 0
END_INDEX = 10

def get_address(row, address):
    address = address.replace("#", "")
    url = 'https://nominatim.openstreetmap.org/search.php?q=' + address.replace(" ", "+") + "&format=json"
    response = requests.get(url).json()
    return response

for index, row in df.iterrows():
     if(index >= START_INDEX and index < END_INDEX):
        try:
          address = row['locn_addr'].strip() + " " + row['locn_cty'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
          response = get_address(row, address)

          if not response:
            address = row['locn_cty'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(row, address)

          if not response:
            address = row['locn_addr'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(row, address)

          if not response:
            address = row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(row, address)

          if(response):
            df.at[index, 'latitude'] = response[0]['lat']
            df.at[index, 'longitude'] = response[0]['lon']
            df.at[index, 'display_name'] = response[0]['display_name']
          
          zipcode = search.by_zipcode(row['zip_cd'])
          # Checking for non std postal codes
          if not zipcode.population:
              # Checking for non std zipcodes like postal boxes
              res = search.by_city_and_state(city=zipcode.major_city, state=zipcode.state)
              if (len(res)) > 0:
                  zipcode = res[0]

          df.at[index, 'population_density'] = zipcode.population_density
          df.at[index, 'median_household_income'] =  zipcode.median_household_income

        except Exception as e:
            print("An exception occurred:", e)
            continue

df['normalised_population_density'] = (((df['population_density'] - df['population_density'].min()) / (df['population_density'].max() - df['population_density'].min())) * 100)
df['normalised_economy'] = (((df['median_household_income'] - df['median_household_income'].min()) / (df['median_household_income'].max() - df['median_household_income'].min())) * 100)

df.head()

Exception ignored in: <function SearchEngine.__del__ at 0x7f26fa21ae50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 196, in __del__
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 202, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1811, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1853, in _close_impl
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 923, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2416, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2639, in _do_close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2625, in _close_impl
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2617, in _connection_rollback_impl
  File "/usr/lo

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,median_household_income,normalised_population_density,normalised_economy
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W,31.7784751,-106.38271330797117,"Cinemark, 8401, Gateway Boulevard West, El Pas...",2402.0,44217.0,7.918473,12.049769
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD,37.9647421,-122.0603194,"Sears Auto Center, 1001, Sunvalley Boulevard, ...",3688.0,47313.0,14.327005,15.094915
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN,33.9457006,-117.9995233,"Whittwood Lane, Whittwood Town Center, Friendl...",6136.0,90477.0,26.526137,57.549916
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL,47.4627356,-122.2559156,"Tukwila, King County, Washington, United States",3062.0,40815.0,11.207455,8.703649
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST,42.22089312121212,-71.02840827272726,"250, Granite Street, Braintree, Norfolk County...",2600.0,81844.0,8.905168,49.058719


In [116]:
df.isnull().sum()

locn_nbr                          0
locn_cty                          0
locn_st_cd                        0
zip_cd                            0
zip_pls_4                         0
locn_addr                         0
latitude                         89
longitude                        89
display_name                     89
population_density               89
median_household_income          89
normalised_population_density    89
normalised_economy               89
dtype: int64

In [117]:
# The haversine formula is used to  calculate the distance between two geographic coordinates on the earth. 
# This formula calculates the great-circle distance between two points on a sphere given their longitudes and latitudes.

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    # Calculate the haversine distance
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of the earth in kilometers
    return c * r

## Calculating distance from nearest Walmart

In [118]:
# Load walmart stores location data
walmart_stores_data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/walmartstoreloaction.csv")

for index, row in df.iterrows():
    distances_to_walmarts = []
    if(index >= START_INDEX and index < END_INDEX):
        for idx, walmart_store in walmart_stores_data.iterrows():
          distances_to_walmarts.append(haversine_distance(float(row['latitude']), float(row['longitude']), float(walmart_store['latitude']), float(walmart_store['longitude'])))
        min_dist = min(distances_to_walmarts)
        df.at[index, 'distance_from_nearest_walmart'] = min_dist

## Calculating distance from nearest Costco

In [119]:
# Load costco stores location data
costco_stores_data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/costco_store.csv")

for index, row in df.iterrows():
    distances_to_costcos = []
    if(index >= START_INDEX and index < END_INDEX):
        for idx, costco_store in costco_stores_data.iterrows():
          distances_to_costcos.append(haversine_distance(float(row['latitude']), float(row['longitude']), float(costco_store['latitude']), float(costco_store['longitude'])))
        min_dist = min(distances_to_costcos)
        df.at[index, 'distance_from_nearest_costco'] = min_dist

In [123]:
# Weightage for each parameter
DISTANCE_WEIGHT = 0.4
POPULATION_WEIGHT = 0.3
ECONOMY_WEIGHT = 0.3

NEAREST_WALMART_DISTANCE_WEIGHT = 0.6
NEAREST_COSTCO_DISTANCE_WEIGHT = 0.4

df['normalised_distance_from_nearest_walmart'] = (((df['distance_from_nearest_walmart'] - df['distance_from_nearest_walmart'].min()) / (df['distance_from_nearest_walmart'].max() - df['distance_from_nearest_walmart'].min())) * 100)
df['normalised_distance_from_nearest_costco'] = (((df['distance_from_nearest_costco'] - df['distance_from_nearest_costco'].min()) / (df['distance_from_nearest_costco'].max() - df['distance_from_nearest_costco'].min())) * 100)

In [124]:
df.head()

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,median_household_income,normalised_population_density,normalised_economy,distance_from_nearest_walmart,distance_from_nearest_costco,normalised_distance_from_nearest_walmart,normalised_distance_from_nearest_costco,score,normalised_score,rank
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W,31.7784751,-106.38271330797117,"Cinemark, 8401, Gateway Boulevard West, El Pas...",2402.0,44217.0,7.918473,12.049769,0.213845,2.716829,0.000000,8.374915,7.047284,90.455262,3.0
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD,37.9647421,-122.0603194,"Sears Auto Center, 1001, Sunvalley Boulevard, ...",3688.0,47313.0,14.327005,15.094915,5.597910,2.336812,78.187282,3.657004,-33.587420,29.381982,6.0
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN,33.9457006,-117.9995233,"Whittwood Lane, Whittwood Town Center, Friendl...",6136.0,90477.0,26.526137,57.549916,3.274059,3.064878,44.440363,12.695936,13.397813,100.000000,1.0
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL,47.4627356,-122.2559156,"Tukwila, King County, Washington, United States",3062.0,40815.0,11.207455,8.703649,2.819904,2.042249,37.845134,0.000000,-13.001909,60.321660,5.0
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST,42.22089312121212,-71.02840827272726,"250, Granite Street, Braintree, Norfolk County...",2600.0,81844.0,8.905168,49.058719,3.862893,10.097019,52.991402,100.000000,-38.797543,21.551255,8.0


## Generate the final score for each location

In [125]:
for index, row in df.iterrows():
     if(index >= START_INDEX and index < END_INDEX):
        try:
          # Assigning a score based on greatest population density, higher economy, and low distances from Walmart and Costco
          score_of_distance_from_stores = (df.at[index, 'normalised_distance_from_nearest_walmart'] * NEAREST_WALMART_DISTANCE_WEIGHT + df.at[index, 'normalised_distance_from_nearest_costco'] * NEAREST_COSTCO_DISTANCE_WEIGHT) * DISTANCE_WEIGHT
          df.at[index, 'score'] =  df.at[index, 'normalised_population_density'] * POPULATION_WEIGHT + df.at[index, 'normalised_economy'] * ECONOMY_WEIGHT - score_of_distance_from_stores

        except Exception as e:
            print("An exception occurred", e)
            continue

df['normalised_score'] = (((df['score'] - df['score'].min()) / (df['score'].max() - df['score'].min())) * 100)

df['rank'] = df['normalised_score'].rank(ascending=False)
df.head()

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,median_household_income,normalised_population_density,normalised_economy,distance_from_nearest_walmart,distance_from_nearest_costco,normalised_distance_from_nearest_walmart,normalised_distance_from_nearest_costco,score,normalised_score,rank
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W,31.7784751,-106.38271330797117,"Cinemark, 8401, Gateway Boulevard West, El Pas...",2402.0,44217.0,7.918473,12.049769,0.213845,2.716829,0.000000,8.374915,4.650486,57.775388,3.0
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD,37.9647421,-122.0603194,"Sears Auto Center, 1001, Sunvalley Boulevard, ...",3688.0,47313.0,14.327005,15.094915,5.597910,2.336812,78.187282,3.657004,-10.523492,23.497412,6.0
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN,33.9457006,-117.9995233,"Whittwood Lane, Whittwood Town Center, Friendl...",6136.0,90477.0,26.526137,57.549916,3.274059,3.064878,44.440363,12.695936,12.525779,75.565653,2.0
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL,47.4627356,-122.2559156,"Tukwila, King County, Washington, United States",3062.0,40815.0,11.207455,8.703649,2.819904,2.042249,37.845134,0.000000,-3.109501,40.245598,5.0
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST,42.22089312121212,-71.02840827272726,"250, Granite Street, Braintree, Norfolk County...",2600.0,81844.0,8.905168,49.058719,3.862893,10.097019,52.991402,100.000000,-11.328770,21.678291,7.0
